In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bz2 #can handle .bz2 compressed files

# Exercise 1

## Problem statement
*Count the number of lines in Python for each file.*

## Solution

In [2]:
#Filenames declaration
fsearches='searches.csv.bz2'
fbookings='bookings.csv.bz2'

In [ ]:
# Read the file line by line and count
count=0
with bz2.BZ2File(fsearches,"r") as file:
    for line in file:
        count +=1
print count
nlines_searches = count
# Read the file line by line and count
count=0
with bz2.BZ2File(fbookings,"r") as file:
    for line in file:
        count +=1
print count
nlines_bookings=count

In [7]:
print "File %s has %s lines."%(fsearches,nlines_searches)
print "File %s has %s lines."%(fbookings,nlines_bookings)

File searches.csv.bz2 has 20390199 lines.
File bookings.csv.bz2 has 10000011 lines.


# Exercise 2

## Problem statement

*top 10 arrival airports in the world in 2013 (using the bookings file)*

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)

## Solution

### 1) Accessing data efficiently

Prepare dataframe to be read using chunks (due to RAM restrictions). Note that the columns are separated by '^' character in the Bookings file.

We will define two different chunksizes:

    * chksize_big: for production
    * chksize_small: for testing and overview purposes.
    
Read column labels and sample (first chunk) to overview data.

In [3]:
chksize_big=1000000 #used for production
chksize_small = 1000 #used for testing
reader = pd.read_csv(fbookings,compression='bz2', sep='^',iterator=True, chunksize=chksize_small)
#Get some relevant information about the dataframe
for df in reader:
    [rows,cols] = df.shape
    bookings_labels = df.columns
    bookings_head   = df.head()
    bookings_tail   = df.tail()
    break
#Show sample
print bookings_head
print bookings_tail

   act_date             source  pos_ctry                          pos_iata  \
0  2013-03-05 00:00:00  1A      DE        a68dd7ae953c8acfb187a1af2dcbe123   
1  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
2  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
3  2013-03-26 00:00:00  1A      AU        0f984b3bb6bd06661c95529bbd6193bc   
4  2013-03-26 00:00:00  1A      AU        0f984b3bb6bd06661c95529bbd6193bc   

                          pos_oid                      rloc            \
0  1a11ae49fcbf545fd2afc1a24d88d2b7  ea65900e72d71f4626378e2ebd298267   
1  7437560d8f276d6d05eeb806d9e7edee  737295a86982c941f1c2da9a46a14043   
2  7437560d8f276d6d05eeb806d9e7edee  737295a86982c941f1c2da9a46a14043   
3  36472c6dbaf7afec9136ac40364e2794  5ecf00fdcbcec761c43dc7285253d0c1   
4  36472c6dbaf7afec9136ac40364e2794  5ecf00fdcbcec761c43dc7285253d0c1   

   cre_date             duration  distance  dep_port    ...      \
0  2013-02-22 00:00:00   

In [4]:
print bookings_labels

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')


### 2) Filtering dataframe

Reading the dataframe in chunks, we will append the target columns on a new dataframe (written as hdf5 file called **'ex2.h5'**). 
We need to pay special attention to missing values. In this case we have found some 'Na' value on PAX, this giving errors when trying to append the data.

In [2]:
#Filename to write the filtered results
ffilter = 'ex2.h5'

In [218]:
#In order to handle numerica NaN values on PAX, we will set missing values to 0.
def nan2zero(num):
    try:
        return np.int(num)
    except:
        return 0

reader = pd.read_csv(fbookings,compression='bz2', sep='^',iterator=True,memory_map=True, \
                     chunksize=chksize_big,converters={'pax':nan2zero})
with pd.HDFStore(ffilter,mode='w') as store:
    nc = 1
    for df in reader:
        print "Reading chunk number %s"%nc
        #Reduce the problem to two columns
        df2            = df.filter(['arr_port','pax'],axis=1)
        df2['pax']     = df2['pax'].astype(int)
        df2['arr_port']= df2['arr_port'].astype('str')
        #Write to file
        store.append('bookings', df2)#data_columns=['arr_port','pax'])
        nc += 1

Reading chunk number 1
Reading chunk number 2
Reading chunk number 3
Reading chunk number 4
Reading chunk number 5
Reading chunk number 6
Reading chunk number 7
Reading chunk number 8
Reading chunk number 9
Reading chunk number 10
Reading chunk number 11


Now, the problem has been reduced to a dataframe with only 2 columns (instead of 38) with a size <200 Mb.

### 3) Find top 10 airports by pax numbers.

In [3]:
#Read the filtered file and check dataframe
df = pd.read_hdf(ffilter,mode='r')
df.tail(10)

,arr_port,pax
10000000,MEL,2
10000001,LAS,2
10000002,LAS,2
10000003,MAA,1
10000004,MAA,1
10000005,MAA,-1
10000006,MAA,1
10000007,RDU,1
10000008,ORY,1
10000009,ORY,1


** We are assuming that all the information provided in arr_port and pax is correct. Moreover, we will eliminate now the airports with 'NaN', without trying to infer the value by inspection.**

First, we will make sure that IATA code is given by the 3 first characters without any space behind (trying to avoid possible duplicates).

In [4]:
df['arr_port']=df['arr_port'].str[0:3]

Note that **there exists an airport with code NAN**, but all the IATA codes are given with Capital letters, 
so that, it is expected that the value 'NaN' is just a missing value.

In [5]:
mask = df['arr_port'].isin(['NaN', 'NA', ' '])
print df.shape
df = df[~mask]
print df.shape

(10000010, 2)
(10000010, 2)


No missing value has been found on the data.

Now, group by arr_port and sum the numbers on pax column

In [6]:
df2 = df.groupby(['arr_port']).sum()

In [7]:
print df2.head(5)
print df2.tail(5)

           pax
arr_port      
AAB         30
AAE        810
AAL        810
AAN         20
AAQ       1650
           pax
arr_port      
ZWE         30
ZWS        620
ZYL       1290
ZYN         20
ZYR       1020


Sort in descending order and show the **top 10 airports**:

In [8]:
df3=df2.sort_values('pax',ascending=False)
df3.head(10)

,pax
arr_port,
LHR,88809
MCO,70930
LAX,70530
LAS,69630
JFK,66270
CDG,64490
BKK,59460
MIA,58150
SFO,58000


### 4) Find the names of the cities using GeoBases

First, we need to clone and follow the instructions for installation of GeoBases. Once it is installed, we can use it in order to give the name of the city for a given Airport code.

In [29]:
from GeoBases import GeoBase
geo_o = GeoBase(data='ori_por', verbose=False)
geo_a = GeoBase(data='airports', verbose=False)

In [31]:
#Testing geobases with an example
geo_o.get('LHR','city_name_ascii')

'London'

Take the previous dataframe with the top 10 airports and convert it to lists.

In [73]:
top10 = df3.head(10)
top10 = top10.reset_index(level=['arr_port']) #Reset index in order to use arr_port as a column again
top10_names = top10['arr_port'].tolist()
top10_pax = top10['pax'].tolist()
print top10_names

['LHR', 'MCO', 'LAX', 'LAS', 'JFK', 'CDG', 'BKK', 'MIA', 'SFO', 'DXB']


In [77]:
# Print results nicely
print "Top 10 ranking airports/cities"
ipos = 1
for IATA in top10_names:
    city = geo_o.get(IATA,'city_name_ascii')
    pax = top10_pax[ipos-1]
    print "%s) %s, city: %s --> %s pax"%(ipos,IATA,city,pax)
    ipos += 1

Top 10 ranking airports/cities
1) LHR, city: London --> 88809 pax
2) MCO, city: Orlando --> 70930 pax
3) LAX, city: Los Angeles --> 70530 pax
4) LAS, city: Las Vegas --> 69630 pax
5) JFK, city: New York City --> 66270 pax
6) CDG, city: Paris --> 64490 pax
7) BKK, city: Bangkok --> 59460 pax
8) MIA, city: Miami --> 58150 pax
9) SFO, city: San Francisco --> 58000 pax
10) DXB, city: Dubai --> 55590 pax


### 5) Sanity check

After all the data manipulation it is importat to check if the results make sense.

First of all we see that the top 10 airports are indeed important hubs or big cities. In addition, the pax numbers are also reasonable. We can also see that Geobase package is working well; for instance: LHR, LAX and JFK are some well-known aiports.